In [1]:
!pip3 install transformers==3.1.0
!gdown --id 1-di0q_8O1_QsOSEm1sIOQWvuSy2LTIYx
!tar -xzvf bert_lex_ktrain_20201221.tar.gz
!pip3 install git+https://github.com/Vakihito/ktrain.git
!pip3 install pytube
!git clone https://github.com/oarriaga/face_classification

Defaulting to user installation because normal site-packages is not writeable
Downloading...
From: https://drive.google.com/uc?id=1-di0q_8O1_QsOSEm1sIOQWvuSy2LTIYx
To: /home/akihito/Desktop/TCC/bert_lex_ktrain_20201221.tar.gz
526MB [01:30, 5.79MB/s] 
bert_lex_multilingual_small.model/
bert_lex_multilingual_small.model/tf_model.preproc
bert_lex_multilingual_small.model/tf_model.h5
bert_lex_multilingual_small.model/config.json
df_lex_embeddings.pickle
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/Vakihito/ktrain.git to /tmp/pip-req-build-triy9oup
  Running command git clone -q https://github.com/Vakihito/ktrain.git /tmp/pip-req-build-triy9oup
Defaulting to user installation because normal site-packages is not writeable
Cloning into 'face_classification'...
remote: Enumerating objects: 990, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 990 (delta 1), reused 0 (delt

In [2]:
img_feat_size = 112
text_feat_size = 768

In [3]:
import cv2
from pytube import YouTube
from pytube.helpers import safe_filename
from statistics import mode
import os
from keras import backend as K


import cv2
from keras.models import load_model
import numpy as np
from textblob import TextBlob


from face_classification.src.utils.datasets import get_labels
from face_classification.src.utils.inference import detect_faces
from face_classification.src.utils.inference import draw_text
from face_classification.src.utils.inference import draw_bounding_box
from face_classification.src.utils.inference import apply_offsets
from face_classification.src.utils.inference import load_detection_model


# Bert

Carregando a modalidade de texto


In [5]:
import keras
import ktrain
from ktrain import text
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from keras import backend as K

predictor_bert_lex = ktrain.load_predictor('bert_lex_multilingual_small.model')
df_lex  = pickle.load(open('df_lex_embeddings.pickle','rb'))

In [6]:
def get_embedding(text,predictor):
  tokenizer = predictor.preproc.get_tokenizer()
  input_ids = tf.constant(tokenizer.encode(text))[None, :]  # Batch size 1
  outputs = predictor.model.layers[0](input_ids)
  last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
  v = np.average(last_hidden_states.numpy()[0],axis=0)
  return v

def get_bert_lex(text,k=3,min_query=0.8,use_predictor=False):
  global df_lex, predictor_bert_lex

  sentiment_model = predictor_bert_lex.predict(text)
  features = get_embedding(text,predictor_bert_lex)
  a = np.array([features])
  b = np.array(list(df_lex['embedding']))
  cos_sim = cosine_similarity(a,b)
  df_lex['query'] = cos_sim[0]

  temp = df_lex.sort_values(by='query',ascending=False).head(k)
  if (temp.iloc[0]['query'] > min_query ):
    temp = temp[temp['query'] > min_query]
  sentiment_lex = np.average(np.array(temp['sentiment'].to_list()))
  if use_predictor:
    return temp, sentiment_lex, sentiment_model
  return temp, sentiment_lex

text= 'pessoas brigando na rua'
df_explain, sentiment_lex = get_bert_lex(text,k=7)
print(sentiment_lex)
df_explain.head(1)

-1.0


,text,lang,sentiment,embedding,query
2383,picketing,en,-1,"[0.22929063, 0.15071137, -0.67776394, -0.17623...",0.95456


# Caption

Definindo algumas funções para tratamento das captions geradas pelo Pytube

In [7]:
# gets the string time as HH:MM:SS and return in seconds
def string_time_int(str_time):
  segundos = int(str_time[-2:])
  segundos += 60 * int(str_time[-5:-3])
  segundos += 3600 * int(str_time[:-6])
  return segundos
def process_caption(video, lang='a.en'):
  '''
    extract the information from the caption and saves in a caption struct 
  '''
  has_caption = False
  print(video.captions)
  for cap in video.captions:
    if cap.code == lang:
      has_caption = True

  if not has_caption:
    print("caption of lang : " + lang + " not found")

  video_len = video.length

  captions  = video.captions[lang]
  captions_str = str(captions.generate_srt_captions())
  list_captions = []
  line_counter = 1
  for line in captions_str.split('\n'):
      # time type
      if (line_counter + 2) % 4  == 0:
          line_aux = line.split(" --> ")
          time_s = string_time_int(line_aux[0][:-4])
          time_e = string_time_int(line_aux[1][:-4])
      # comment type
      if (line_counter + 1) % 4  == 0 and video_len >= time_e:  
          list_captions.append([line, time_s, time_e])
      line_counter += 1

  return list_captions

In [8]:
def my_preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

def predict_text(text):
    global predictor_bert_lex
    polarity_text = predictor_bert_lex.predict(text)

    color = np.asarray((255, 255, 255))

    if polarity_text < -0.4:
        color = abs(polarity_text) * np.asarray((255, 0, 0))
    elif polarity_text > 0.4:
        color = abs(polarity_text) * np.asarray((0, 255, 0))
    
    return color, polarity_text



# parameters for loading data and images
detection_model_path = './face_classification/trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = './face_classification/trained_models/emotion_models/fer2013_mini_XCEPTION.51-0.63.hdf5'
emotion_labels = get_labels('fer2013')

# Carregando os modelos de imagem

In [9]:
# loading models
face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)


### gets the embedding from the emtion_classifier
def getFaceEmbedding(gray_face,layer_out=3):
  global emotion_classifier
  inp = emotion_classifier.input

  functor1 = K.function([inp], emotion_classifier.layers[-layer_out].output)
  emotion_prediction = functor1([gray_face])[0]

  return emotion_prediction.flatten()

# Fazendo download do vídeo
Utilizamos o pytube para extrair a resolução do vídeo. Nesse sentido, os seguintes parâmetros são importante:
 - vid_id - é o id do vídeo do youtube que desejamos extrair
 - res - é a resulução do vídeo que desejamos extrair - creio que seja melhor manter 480, para o bom funcionamento das redes neurais
 - lang - é lingua da caption, indicaria utilizar 'en', mas caso queira utilizar legendas automáticas utilize 'a.en'

In [10]:
# hyper-parameters for bounding boxes shape
frame_window = 10
emotion_offsets = (20, 40)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []

# starting video streaming

vid_id = "xiQ38WKoF2E"
res = 480
lang = 'en'


url = "https://www.youtube.com/watch?v=" + vid_id
print("downloading the video from the ulr : ", url)
video = YouTube(url)

max_res = -1
itag_max = -1

fps_my = 30
for stream in video.streams:
    if stream.resolution and stream.mime_type == "video/mp4":
      current_res = int(stream.resolution[:-1])
      current_fps = int(stream.fps)
      if current_res <= res and max_res < current_res:
        max_res = current_res
        itag_max = stream.itag
        fps_my =  float(current_fps) 

if max_res == -1:
  print("erro : chose another resolution")

font_scale_cur = max_res/144

print(str(itag_max))
video.streams.get_by_itag(str(itag_max)).download()

dirname = './'
video_path = os.path.join(dirname, safe_filename(video.title) + '.mp4')


#documentation: https://pypi.org/project/pafy/
# video_capture = cv2.VideoCapture(video_path)


captions_list = process_caption(video,'en')
captions_list.sort(key=lambda x: x[1])


time = 0
frame_counter = 0
caption_counter = 0

video_frequency = 1

downloading the video from the ulr :  https://www.youtube.com/watch?v=xiQ38WKoF2E
135
{'en': <Caption lang="English" code="en">, 'a.en': <Caption lang="English (auto-generated)" code="a.en">}


In [11]:
# removendo certos caracteres da caption
for i in range(len(captions_list)):
  captions_list[i][0] = captions_list[i][0].replace(u'\xa0', u' ')

In [12]:
# definindo um dicionario que relacionará os valores
video_dict_all = {
    'frame' : [],
    'time' : [],
    'caption' : [],
    'caption_polarity' : [],
    'caption_emb' : [],
    'face' : [],
    'face_polarity' : [],
    'face_emb' : []

}

In [13]:
def fill_dict(frame,time,
              caption, caption_polarity, caption_emb,
              face, face_polarity, face_emb):
  video_df_all = {
    'frame' : [frame],
    'time' : [time],
    'caption' : caption,
    'caption_polarity' : caption_polarity,
    'caption_emb' : caption_emb,
    'face' : face,
    'face_polarity' : face_polarity,
    'face_emb' : face_emb
  }

  if len(caption) == 0:
    video_df_all['caption'] = ['-1']
    video_df_all['caption_polarity'] = [0]
    video_df_all['caption_emb'] = [np.zeros(text_feat_size)]

  if len(face) == 0:
    video_df_all['face'] = ['-1']
    video_df_all['face_polarity'] = [0]
    video_df_all['face_emb'] = [np.zeros(img_feat_size)]
  
  if len(face) >= 2:
    for i in range(len(face) - 1):
      video_df_all['frame'].append(video_df_all['frame'][-1])
      video_df_all['time'].append(video_df_all['time'][-1])
      video_df_all['caption'].append(video_df_all['caption'][-1])
      video_df_all['caption_polarity'].append(video_df_all['caption_polarity'][-1])
      video_df_all['caption_emb'].append(video_df_all['caption_emb'][-1])

  return video_df_all

In [14]:
captions_list_size = len(captions_list)

def return_text(time, counter):
  temp_text = ""
  for i in range(counter, captions_list_size):
    if time >= captions_list[i][1] and time <= captions_list[caption_counter][2]:
      temp_text +=  captions_list[i][0] + " "
    elif captions_list[i][1] > time:
      return  temp_text
  return temp_text

In [15]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

# Loop de processamento
Neste loop processamos o vídeo através dos modelos unimodais
 - frame_limit - o limite de frames a serem analizados, caso queira analizar todos os frames coloque inf.
 - size_shape - formato da imagem a ser gerada
  

In [48]:
frame_limit = 50
size_shape = ( 240 , 320 )
cv2.namedWindow('window_frame')
img_counter = 0
time = 0
frame_counter = 0
caption_counter = 0
video_capture = cv2.VideoCapture(video_path)

# Coordenadas 
# 1 - distancia da borda lateral esquerda 
# 2 - distancia da borda superior 
#

while True:
    success , bgr_image = video_capture.read()
    
    if not success:
        print("fim !")
        break
      
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    faces = detect_faces(face_detection, gray_image)

    time = frame_counter/fps_my 


    caption = []
    caption_polarity = []
    caption_emb = []

    cur_text = return_text(time, caption_counter)
    if len(cur_text) > 0:
        color_caption, caption_polarity = predict_text(cur_text)
        caption = [cur_text]
        caption_polarity = [caption_polarity]
        caption_emb = [get_embedding(cur_text,predictor_bert_lex)]
        if frame_counter % video_frequency == 0:
        
          print("caption polarity:", caption_polarity)
          print("caption sentence:", cur_text)

        color_caption = color_caption.tolist()
        draw_text(np.array([10,max_res - 30,50,50]), rgb_image, cur_text,color_caption,font_scale=0.4, thickness=1)
        while caption_counter < len(captions_list) - 1 and time >= captions_list[caption_counter][2]:  
            caption_counter += 1

    face = []
    face_polarity = []
    face_emb = []
    if len(faces) == 0 :
      print("face not found !")

    for face_coordinates in faces:

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)

        gray_face = gray_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
        except cv2.error:
            print('error : on resize')
            continue

        gray_face = my_preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)

        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)

        emotion_text = emotion_labels[emotion_label_arg]
        emotion_window.append(emotion_text)

        if len(emotion_window) > frame_window:
            emotion_window.pop(0)
        try:
            emotion_mode = mode(emotion_window)
        except:
            continue

        ### threshold wall
        # ysnp = you shall not pass
        ysnp = False
        sentiment = 'neutral'
        if emotion_text == 'angry' and emotion_probability >= 0.5:
            ysnp = True
            sentiment = 'negative'
        if emotion_text == 'happy' and emotion_probability >= 0.7:
            ysnp = True
            sentiment = 'positive'
        if emotion_text == 'sad' and emotion_probability >= 0.45:
            ysnp = True
            sentiment = 'negative'
        if emotion_text == 'neutral' and emotion_probability >= 0.5:
            ysnp = True
            sentiment = 'neutral'
        if emotion_text == 'fear' and emotion_probability >= 0.5:
            ysnp = True
            sentiment = 'negative'
        if emotion_text == 'surprise' and emotion_probability >= 0.4:
            ysnp = True
            sentiment = 'positive'

        if ysnp:
            senti_multi = 0
            if sentiment == 'negative':
                senti_multi = -1
                color = emotion_probability * np.asarray((255, 0, 0))
            elif sentiment == 'positive':
                senti_multi = 1
                color = emotion_probability * np.asarray((0, 255, 0))
            else:
                senti_multi = 0
                color = emotion_probability * np.asarray((0, 0, 255))

            color = color.astype(int)
            color = color.tolist()

            draw_bounding_box(face_coordinates, rgb_image, color)
            draw_text(face_coordinates, rgb_image, sentiment,color, 0, -45, 1, 1)
            if frame_counter % video_frequency == 0:
              print('face polarity: ', emotion_probability * senti_multi,emotion_text)


            face.append('img' + str(img_counter))
            face_polarity.append(emotion_probability * senti_multi)
            face_emb.append(getFaceEmbedding(gray_face) )

            img_counter += 1
        elif frame_counter % video_frequency == 0:
            print("you shall not pass")

   
    dict_temp= fill_dict(frame_counter, time,
                        caption, caption_polarity, caption_emb,
                        face, face_polarity, face_emb)       
    for key in dict_temp.keys():
      video_dict_all[key] += dict_temp[key]
    
    frame_counter += 1

  # if len(cur_text) > 0:

#     if frame_counter % video_frequency == 0:
        
    print("fc : ",frame_counter)
    resized = cv2.resize(rgb_image, size_shape)
    bgr_image = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    print("#" * 50)
    if(frame_counter > frame_limit):
        break

video_capture.release()
cv2.destroyAllWindows()

caption polarity: [0.7204882]
caption sentence: Today we are going to 
face polarity:  0.0 neutral
fc :  1
##################################################
caption polarity: [0.7204882]
caption sentence: Today we are going to 
face polarity:  0.0 neutral
fc :  2
##################################################
caption polarity: [0.7204882]
caption sentence: Today we are going to 
face polarity:  0.0 neutral
fc :  3
##################################################
caption polarity: [0.7204882]
caption sentence: Today we are going to 
face polarity:  0.0 neutral
fc :  4
##################################################
caption polarity: [0.7204882]
caption sentence: Today we are going to 
face polarity:  0.0 neutral
fc :  5
##################################################
caption polarity: [0.7204882]
caption sentence: Today we are going to 
you shall not pass
fc :  6
##################################################
caption polarity: [0.7204882]
caption sentence: Today we are 

# Correlação entre caption e text
 - time  - mostra o tempo em segundos de início da análise
 - frame - mostra o frame sobre análise
 - caption - mostra a legenda mostrada
 - caption_polarity - mostra a polaridade da caption
 - caption_emb - mostra a embedding gerada pelo Bert
 - face - mostra um couter referente a imagem
 - face_polarity - mostra a polaridade da face
 - face_emb - mostra a embedding gerada pelo modelo de faces

In [18]:
import pandas as pd

video_df = pd.DataFrame(video_dict_all)

video_df.head()

,frame,time,caption,caption_polarity,caption_emb,face,face_polarity,face_emb
0,0,0.000000,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img0,0.0,"[-1.9817237, -7.3088036, -0.21659227, 1.669573..."
1,1,0.033333,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img1,0.0,"[-2.35278, -7.1067867, -0.80641127, 3.150189, ..."
2,2,0.066667,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img2,0.0,"[-1.0932013, -4.5289273, 1.1658401, 1.031141, ..."
3,3,0.100000,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img3,0.0,"[-4.0827923, -7.5882707, -0.90062743, 3.067546..."
4,4,0.133333,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img4,0.0,"[-2.856795, -5.6585393, 0.58233565, 1.700731, ..."


# Gerando concordância entre modelos
Neste passo preparamos o modelo para treino

In [19]:
def concord_bool(a,b):
  multi = a * b
  if multi >= 0:
    return True
  return False

def concord_val(a,b):
  (pa , pb) = (a,b)
  if abs(a) < 0.3:
    pa = 0
  if abs(b) < 0.3:
    pb = 0
  multi = pa * pb
  if multi >= 0:
    if pa < 0 or pb < 0:
      return 'negative'
    if pa > 0 or pb > 0:
      return 'positive'
  return 'neutral'

def return_dict():
  dict_aux = {
    'frame' : [],
    'time' : [],
    'caption' : [],
    'caption_polarity' : [],
    'caption_emb' : [],
    'face' : [],
    'face_polarity' : [],
    'face_emb' : [],
    'intent' : []
  }
  return dict_aux

In [20]:
def get_concord_embedding():
  global video_df
  couter = 0
  df_train = return_dict()
  df_test = return_dict()
  concord = []
  for i , row in video_df.iterrows():
    if concord_bool(row['face_polarity'],row['caption_polarity']):
      for key in row.keys():
        df_train[key].append(row[key])
      df_train['intent'].append(concord_val(row['face_polarity'],row['caption_polarity']))
      concord.append(1)
    else:
      for key in row.keys():
        df_test[key].append(row[key])
      df_test['intent'].append(concord_val(row['face_polarity'],row['caption_polarity']))
      concord.append(0)
  video_df['consent'] = concord
  return df_train, df_test


In [21]:
(df_train, df_test) =  get_concord_embedding()
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [23]:
import tensorflow as tf
ACCURACY_THRESHOLD = 0.95
class acc_callback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            self.model.stop_training = True

# Definindo um modelo
Para esta aplicação definimos um modelo de atenção em relação a modalidade 

In [24]:
from tensorflow.keras.layers import Dot,Activation,Dense, Input, concatenate, multiply, average, subtract, add, Dropout, Lambda, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf


def multimodal_text_image(num_classes,size_1=img_feat_size,size_2=text_feat_size,operator='att',verbose=0):

  # fusion_dim = X_1.shape[1]+X_2.shape[1]
  fusion_dim = size_1

  inp1 = Input(shape=size_1)
  inp2 = Input(shape=size_2)

  l1 = Dense(fusion_dim, activation='relu')(inp1)
  l2 = Dense(fusion_dim, activation='relu')(inp2)

  if(operator=='att'):
    visual_embd = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1))(l1) # (bs, ndim)
    average_seq = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1))(l2) # (bs, ndim)
    scalar_visual = Dense(1)(visual_embd) # (bs, 1)
    scalar_text = Dense(1)(average_seq) # (bs, 1)
    scalars = concatenate([scalar_visual, scalar_text], name='concat')  # (bs, 2)

    # # Step 2. Normalize weights - softmax
    alphas = Activation('softmax')(scalars) # (bs, 2)

    # Step 3. Weighted average
    visual_embd_2 = Lambda( lambda x: tf.keras.backend.expand_dims(x) ) (visual_embd) # (bs, ndim, 1)
    average_seq_2 = Lambda( lambda x: tf.keras.backend.expand_dims(x) )(average_seq) # (bs, ndim, 1)
    features = concatenate([visual_embd_2, average_seq_2], name='concat_feats') # (bs, ndim, 2)
    w = Dot(axes=[-1, -1])([alphas, features]) # (bs, ndim)

  w = Dropout(0.5)(w)
  # fusion_layer = Dense(fusion_dim, activation='relu')(w)
  fusion_layer = w

  if (operator == 'att_labels'): # nm: new
    output = Dense(1)(fusion_layer)  # (batch_size, nb_labels, 1)  
    output = Lambda(lambda x: tf.keras.backend.squeeze(x, axis=-1))(output)  # (batch_size, nb_labels)
    output = Activation('softmax')(output)  # (batch_size, nb_labels)    
  else:
    output = Dense(num_classes,activation='softmax')(fusion_layer)

  model = Model(inputs=[inp1, inp2], outputs=output)

  model.compile(optimizer='adam', loss='categorical_crossentropy')
  model.summary()

  return model, fusion_layer



# Treinando o modelo

In [25]:
import keras 
from tqdm.notebook import tqdm

target = 'intent'

merging_layers = ['autoencoder', 'att_labels','att','concatenate','add','subtract','average','multiply']

num_classes = len(df_train[target].unique())
operator = "att"
# Gerando os modelos
my_model, _ = multimodal_text_image(num_classes,operator=operator)




# merging_layers = ['att', 'att_labels'] # test
results = []



(df_data_train, df_data_test) = (df_train, df_test)

total_rows = len(df_data_train[target])

# modalidade da imagem
X_1 = np.array(df_data_train['face_emb'].to_list())
# modalidade do texto
X_2 = np.array(df_data_train['caption_emb'].to_list())

Y = np.array(pd.get_dummies(df_data_train[target]))

num_classes = len(df_data_train[target].unique())

# modalidades imagem e texto do teste
X_1_test = np.array(df_data_test['face_emb'].to_list())
X_2_test = np.array(df_data_test['caption_emb'].to_list())

acc_call = acc_callback()


history = my_model.fit([X_1,X_2], Y,
                  epochs=50,
                  batch_size=16,
                  shuffle=True,verbose=True)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 768)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 112)          12656       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 112)          86128       input_2[0][0]                    
______________________________________________________________________________________________

In [39]:
y_pred = np.array([])
if len(X_1_test) > 0:
  probs = my_model.predict([X_1_test,X_2_test])
  y_pred = np.argmax(probs,axis=1)
  print("size, test case : ", len(X_1_test))
else:
  print("\033[93mnot a single contradiction between the unmodal models\033[0m")

not a single contradiction between the unmodal models


In [27]:
dict_y_pred = {
    0 : 'negative', 
    1 : 'neutral',
    2 : 'positive'
}

In [28]:
y_pred_value = [dict_y_pred[pred] for pred in y_pred]


In [29]:
index = video_df[video_df['consent'] == 0].index


In [30]:
y_pred_value_size = len(y_pred_value)

tuple_idx_val = []

for i in range(y_pred_value_size):
  tuple_idx_val.append((index[i], y_pred_value[i]))

In [31]:
video_consent_df = video_df[video_df['consent'] == 1]

for idx, row in video_consent_df.iterrows():
  tuple_idx_val.append((idx, concord_val(row['caption_polarity'],row['face_polarity']) ))

In [32]:
tuple_idx_val.sort(key=lambda x:x[0])
final_pred = [i[1] for i in tuple_idx_val]


video_df['final_pred'] = final_pred

# Resultados do modelo multimodal
 - consent - consentimento entre modelo de faces e de texto - 1 - verdadeiro, 0 - falso 
 - final_pred - previsão final 

In [33]:
video_df.head()

,frame,time,caption,caption_polarity,caption_emb,face,face_polarity,face_emb,consent,final_pred
0,0,0.000000,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img0,0.0,"[-1.9817237, -7.3088036, -0.21659227, 1.669573...",1,positive
1,1,0.033333,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img1,0.0,"[-2.35278, -7.1067867, -0.80641127, 3.150189, ...",1,positive
2,2,0.066667,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img2,0.0,"[-1.0932013, -4.5289273, 1.1658401, 1.031141, ...",1,positive
3,3,0.100000,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img3,0.0,"[-4.0827923, -7.5882707, -0.90062743, 3.067546...",1,positive
4,4,0.133333,Today we are going to,0.720488,"[0.1871648, -0.08017133, 0.8622961, -0.1523684...",img4,0.0,"[-2.856795, -5.6585393, 0.58233565, 1.700731, ...",1,positive


In [34]:
def draw_plus_sign(img, center=(10,10) ):
  start_point = (center[0] - 5, center[1] - 2)
  end_point = (center[0] + 5, center[1] + 2)
  img_r = cv2.rectangle(img, start_point, end_point, (0,255,0), -1)

  start_point = (center[0] - 2, center[1] - 5)
  end_point = (center[0] + 2, center[1] + 5)
  img_r = cv2.rectangle(img_r, start_point, end_point, (0,255,0), -1)

  return img_r

def draw_minus_sign(img, center=(10,10)):
  start_point = (center[0] - 5, center[1] - 2)
  end_point = (center[0] + 5, center[1] + 2)
  img_r = cv2.rectangle(img, start_point, end_point, (255,0,0), -1)

  return img_r

In [35]:
time_arr =video_df['time'].to_list()
final_pred_arr = video_df['final_pred'].to_list()
size_total = len(time_arr)

In [36]:
tuple_time_pred = [(time_arr[i], final_pred_arr[i]) for i in range(size_total)]


# Mostrando o resultado do modelo multimodal na janela
 - sinal positivo significa que a seção apresentou um polaridade positiva
 - sinal negativo significa que a seção apresentou um polaridade negativa

In [46]:
### re inicializando as variáveis.

video_capture = cv2.VideoCapture(video_path)
time = 0
frame_counter = 0
caption_counter = 0
counter_tuple_total = 0 
sentiment_polarity = ''

In [47]:
while True:
    success , bgr_image = video_capture.read()
    
    if not success:
        break
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    time = frame_counter/fps_my 



    if time >= captions_list[caption_counter][1] and time <= captions_list[caption_counter][2]:
        color_caption = (255,255,255)
        draw_text(np.array([10,max_res - 30,50,50]), rgb_image, cur_text,color_caption,font_scale=0.4, thickness=1)

        if caption_counter < len(captions_list) - 1 and time >= captions_list[caption_counter + 1][1]:  
            caption_counter += 1
    
    if time >= tuple_time_pred[counter_tuple_total][0]:
        sentiment_polarity = tuple_time_pred[counter_tuple_total][1]
        if (tuple_time_pred[counter_tuple_total][1] == 'positive') :
          rgb_image = draw_plus_sign(rgb_image)
        elif (tuple_time_pred[counter_tuple_total][1] == 'negative'):
          rgb_image = draw_minus_sign(rgb_image)

        if counter_tuple_total < len(tuple_time_pred) - 1 and time >= tuple_time_pred[counter_tuple_total + 1][0]:  
            counter_tuple_total += 1
    
    
    frame_counter += 1
    
  
    print("fc : ",frame_counter)
    print(sentiment_polarity)
    resized = cv2.resize(rgb_image, size_shape)
    bgr_image = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break
    print("#" * 50)
    if(frame_counter > frame_limit):
        break

video_capture.release()
cv2.destroyAllWindows()

fc :  1
positive
##################################################
fc :  2
positive
##################################################
fc :  3
positive
##################################################
fc :  4
positive
##################################################
fc :  5
positive
##################################################
fc :  6
positive
##################################################
fc :  7
positive
##################################################
fc :  8
positive
##################################################
fc :  9
positive
##################################################
fc :  10
positive
##################################################
fc :  11
positive
##################################################
fc :  12
positive
##################################################
fc :  13
positive
##################################################
fc :  14
positive
##################################################
fc :  15
positive
#########################